In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import json
import scipy.stats as stats
import math

import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms

import vizualizacia_funkcie as visual
import preprocessing_pipeline as prep_pip

# Predspracovanie validacneho datasetu a opis pripadnych zmien

In [2]:
df1 = pd.read_csv("./data/personal_train.csv", index_col=0)
df2 = pd.read_csv("./data/other_train.csv", index_col=0)

X,y = prep_pip.one_proper_df(df1, df2)

C:\Users\makoh\IAU\git_projekt\IAU\preprocessing_pipeline.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_dataset.iloc[0][attr] = not_null.values[0]


In [3]:
PIP = prep_pip.get_preprocessing_pipeline()

In [4]:
preprocessed_train_data = PIP.fit_predict(X,y)

In [5]:
print(preprocessed_train_data[0].shape)
print(preprocessed_train_data[1].shape)

(3263, 33)
(3263,)


In [6]:
preprocessed_train_data[0].columns

Index(['age', 'mean_oxygen', 'std_oxygen', 'kurtosis_oxygen',
       'skewness_oxygen', 'mean_glucose', 'std_glucose', 'kurtosis_glucose',
       'skewness_glucose', 'sex_0', 'sex_1', 'marital-status_0',
       'marital-status_1', 'marital-status_2', 'marital-status_3',
       'relationship_0', 'relationship_1', 'relationship_2', 'relationship_3',
       'occupation_0', 'occupation_1', 'occupation_2', 'occupation_3',
       'occupation_4', 'occupation_5', 'occupation_6', 'occupation_7',
       'occupation_8', 'workclass_0', 'workclass_1', 'education',
       'hours-per-week-cat', 'income'],
      dtype='object')

# Pri praci s validacnym datasetom sa nepouziva fit_predict ale iba predict

In [7]:
df1 = pd.read_csv("./data/personal_valid.csv", index_col=0)
df2 = pd.read_csv("./data/other_valid.csv", index_col=0)

In [8]:
preprocessed_train_data[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3263 entries, 0 to 3932
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   age                 3263 non-null   float64
 1   mean_oxygen         3263 non-null   float64
 2   std_oxygen          3263 non-null   float64
 3   kurtosis_oxygen     3263 non-null   float64
 4   skewness_oxygen     3263 non-null   float64
 5   mean_glucose        3263 non-null   float64
 6   std_glucose         3263 non-null   float64
 7   kurtosis_glucose    3263 non-null   float64
 8   skewness_glucose    3263 non-null   float64
 9   sex_0               3263 non-null   float64
 10  sex_1               3263 non-null   float64
 11  marital-status_0    3263 non-null   float64
 12  marital-status_1    3263 non-null   float64
 13  marital-status_2    3263 non-null   float64
 14  marital-status_3    3263 non-null   float64
 15  relationship_0      3263 non-null   float64
 16  relati

# Manualne vytvorenie a vyhodnotenie rozhodovacich pravidiel pre klasifikaciu

# Natrenovanie a vyhodnotenie klasifikatora s vyuzitim rozhodovacich stromov

# Optimalizacia hyperparametrov

# Vyhodnotenie vplyvu zvolenej strategie riesenia na spravnost klasifikacie